In [1]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" 
# from models import DailyStats
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline         import Pipeline
from sklearn.compose         import ColumnTransformer
from sklearn.impute          import SimpleImputer
from sklearn.preprocessing   import StandardScaler, OneHotEncoder, OrdinalEncoder
from tensorflow.keras import layers, models, callbacks, optimizers, regularizers
from sklearn.model_selection import train_test_split
import tensorflow as tf



load_dotenv()  # reads .env in working directory
db_url = os.getenv("DATABASE_URL")
db_url = "postgres://u76gs2mfn3l2um:p1577d0e7a90ab74ce0a8dcbd5dab809f4240ab92836929df08e328b6c82c5a13@c2hbg00ac72j9d.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d35v2smh8r48et"
print(db_url)
# Heroku uses postgres:// but SQLAlchemy expects postgresql://
if db_url.startswith("postgres://"):
    db_url = db_url.replace("postgres://", "postgresql://", 1)

engine = create_engine(db_url)


postgres://u76gs2mfn3l2um:p1577d0e7a90ab74ce0a8dcbd5dab809f4240ab92836929df08e328b6c82c5a13@c2hbg00ac72j9d.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d35v2smh8r48et


In [17]:
# Example: pull every row from your 'users' table
# show all columns and let pandas auto-width to your terminal/notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# now read and print
df_daily = pd.read_sql("SELECT * FROM daily_stats;", con=engine)

cols = ['stream_name'] + ['total_subscriptions'] + [c for c in df_daily.columns if c != 'stream_name' and c != 'total_subscriptions' and c != 'id']
df_daily = df_daily[cols]

# display
# df_daily[df_daily['stream_name'] == 'shroud'].tail(10)
df_daily.tail(7)



,stream_name,total_subscriptions,stream_date,day_of_week,is_weekend,is_holiday,stream_start_time,days_since_previous_stream,stream_duration,avg_concurrent_viewers,peak_concurrent_viewers,unique_viewers,viewer_growth_rate,total_num_chats,total_chatters,chat_msgs_per_minute,total_emotes_used,unique_emotes_used,followers_start,followers_end,net_follower_change,new_subscriptions_t1,new_subscriptions_t2_t3,resubscriptions,gifted_subs_received,gifted_subs_given,subscription_cancellations,bits_donated,donation_events_count,total_donation_amount,raids_received,raid_viewers_received,polls_run,poll_participation,predictions_run,prediction_participants,game_category,category_changes,title_length,has_giveaway,has_qna,tags,moderation_actions,messages_deleted,timeouts_bans,avg_sentiment_score,positive_negative_ratio,subs_per_avg_viewer,chat_msgs_per_viewer,subs_7d_moving_avg,subs_3d_moving_avg,viewers_3d_moving_avg,day_over_day_peak_change,created_at,gift_subs_bool,min_sentiment_score,max_sentiment_score
399,peach,7,2025-08-18,Monday,False,False,21:32:05.206409,1,434,716.016129,898,191,0.155727,4005,191,9.231156,198,198,23839.0,23883.0,44,2,0,3,2,0,0,100,1,0.0,5,59,0,0,0,0,Just Chatting,2,13,False,False,"[English, Vtuber, ENVtuber, ADHD, Chatty, Loud...",0,0,0,0.85,None,0.009776,20.926702,24.666667,16.600000,None,-63.0,NaT,True,0.40,0.85
400,smallantmom,0,2025-08-19,Tuesday,False,False,08:59:06.576643,1,69,22.985714,36,25,8.000000,411,25,5.957086,17,17,94916.0,94916.0,0,0,0,0,0,0,0,69,1,0.0,0,0,0,0,0,0,Just Chatting,2,14,False,False,"[yesimreallySmallAntsmom, jointhesunshine, Eng...",0,0,0,0.45,None,0.000000,16.440000,2.000000,1.666667,None,-26.0,NaT,False,0.20,0.90
401,obsydyon,9,2025-08-19,Tuesday,False,False,09:18:05.806280,1,409,18.579462,35,37,16.500000,1134,37,2.766777,78,78,660.0,660.0,0,0,0,0,9,0,0,0,0,0.0,3,39,0,0,0,0,Special Events,4,8,False,False,"[TTS, ObsyFans, OnlyChiefos, Ketchup, THIS, Co...",0,0,0,0.30,None,0.484406,30.513514,3.333333,7.000000,None,2.0,NaT,True,0.15,0.85
402,necros,43,2025-08-19,Tuesday,False,False,06:35:05.106797,2,574,5272.719512,7888,3217,253.451613,32513,3217,56.584720,275,275,911228.0,911228.0,0,11,0,31,1,0,0,400,4,0.0,1,228,0,0,0,0,Marvel Rivals,3,13,False,False,"[English, spiderman, blackpanther]",0,0,0,0.75,None,0.008155,10.094956,53.800000,53.000000,None,1109.0,NaT,True,0.10,0.85
403,barbarousking,255,2025-08-19,Tuesday,False,False,08:02:05.369973,1,496,4621.937500,7503,3766,44.750000,47264,3766,95.406573,966,966,230028.0,230028.0,0,4,0,219,32,0,0,28464,52,0.0,0,0,0,0,0,0,Special Events,3,40,False,False,"[BaldWelcome, Kaizo, HardMode, GIGACHAD, Engli...",0,0,0,0.85,None,0.055172,12.550625,115.400000,112.250000,None,4306.0,NaT,True,0.10,0.85
404,smallant,27,2025-08-19,Tuesday,False,False,15:04:06.901551,5,166,1494.319277,2335,678,2335.000000,3175,678,19.071676,134,134,1458705.0,1458705.0,0,4,0,23,0,0,0,300,1,0.0,1,54,0,0,0,0,Donkey Kong Bananza,1,19,False,False,[English],0,0,0,0.90,None,0.018068,4.655325,47.000000,0.000000,None,341.0,NaT,False,0.55,0.85
405,boesschen,43,2025-08-19,Tuesday,False,False,08:07:05.682639,2,583,236.950257,312,162,312.000000,4946,162,8.497814,363,363,8810.0,8810.0,0,1,0,3,39,0,0,2419,9,0.0,3,133,0,0,0,0,Just Chatting,2,13,False,False,"[Vtuber, ENVtuber, English, Chatty, Playingwit...",0,0,0,0.85,None,0.181473,30.530864,19.250000,18.000000,None,-32.0,NaT,True,0.25,0.85


In [74]:
# # DELETE ROWS

# # read the whole table
# df = pd.read_sql("SELECT * FROM live_stream ORDER BY stream_date, stream_start_time", engine)

# # drop the first two rows in-memory
# df2 = df.iloc[:0]

# # overwrite back to the database (be VERY careful—this will replace the table!)
# df2.to_sql("live_stream", engine, if_exists="replace", index=False)


In [3]:
# col_name = "stream_duration"
# new_col_type = "DOUBLE PRECISION"

# alter_sql = f"""
# ALTER TABLE daily_stats
# ADD COLUMN {col_name} {new_col_type}
# ;
# """

# with engine.begin() as conn:
#     conn.execute(text(alter_sql))

In [66]:
# import pandas as pd
# from datetime import date
# from typing import Union


# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

# # now read and print
# daily_sessions = pd.read_sql("SELECT * FROM live_stream;", con=engine)

# def rollup_single_day(
#     df: pd.DataFrame,
#     stream_name: str,
#     target_date: Union[str, date]
# ) -> pd.DataFrame:
#     """
#     Roll up all 5-min snapshots in `df` for the given stream_name on one day INTO ONE ROW.

#     Parameters
#     ----------
#     df : pd.DataFrame
#       raw live_stream snapshots
#     stream_name : str
#       e.g. "thelegendyagami"
#     target_date : str or date
#       e.g. '2025-08-01' or datetime.date(2025,8,1)

#     Returns
#     -------
#     pd.DataFrame
#       one-row DataFrame ready for `to_sql("daily_stats", ...)`
#     """
#     df = df.copy()
#     # 1) filter to our stream
#     df = df[df["stream_name"] == stream_name]
#     if df.empty:
#         raise ValueError(f"No snapshots for stream {stream_name}")

#     # 2) parse and filter to just that calendar day
#     df["stream_date"] = pd.to_datetime(df["stream_date"]).dt.date
#     day = pd.to_datetime(target_date).date()
#     d = df[df["stream_date"] == day]
#     if d.empty:
#         raise ValueError(f"No snapshots for {stream_name} on {day}")

#     # 3) aggregate rules (same as before)...
#     agg = {
#         "stream_name":           "first",
#         "total_subscriptions":   "sum",
#         "day_of_week":           "first",
#         "is_weekend":            "first",
#         "is_holiday":            "first",
#         "stream_start_time":     "first",
#         "days_since_previous_stream": "first",
#         "stream_duration":       "max",
#         "avg_concurrent_viewers":"mean",
#         "peak_concurrent_viewers":"max",
#         "unique_viewers":        "max",
#         "viewer_growth_rate":    "last",
#         "total_num_chats":       "sum",
#         "total_chatters":        "sum",
#         "chat_msgs_per_minute":  "mean",
#         "total_emotes_used":     "sum",
#         "unique_emotes_used":    "sum",
#         "followers_start":       "first",
#         "followers_end":         "last",
#         "net_follower_change":   "last",
#         "new_subscriptions_t1":  "sum",
#         "new_subscriptions_t2_t3":"sum",
#         "resubscriptions":       "sum",
#         "gifted_subs_received":  "sum",
#         "gifted_subs_given":     "sum",
#         "subscription_cancellations":"sum",
#         "bits_donated":          "sum",
#         "donation_events_count": "sum",
#         "total_donation_amount": "sum",
#         "raids_received":        "sum",
#         "raid_viewers_received": "sum",
#         "polls_run":             "sum",
#         "poll_participation":    "sum",
#         "predictions_run":       "sum",
#         "prediction_participants":"sum",
#         "game_category":         "first",
#         "category_changes":      "max",
#         "title_length":          "first",
#         "has_giveaway":          "max",
#         "has_qna":               "max",
#         "moderation_actions":    "sum",
#         "messages_deleted":      "sum",
#         "timeouts_bans":         "sum",
#         "avg_sentiment_score":   "mean",
#         "positive_negative_ratio":"last",
#         "subs_7d_moving_avg":    "last",
#         "subs_3d_moving_avg":    "last",
#         "viewers_3d_moving_avg": "last",
#         "day_over_day_peak_change":"last",
#         # tags, subs_per_avg_viewer, chat_msgs_per_viewer,
#         # created_at, gift_subs_bool handled below
#     }

#     rolled = (
#         d.groupby(["stream_date"], as_index=False)
#          .agg(agg)
#     )

#     # re-union tags
#     rolled_tags = (
#         d.groupby(["stream_date"])["tags"]
#          .apply(lambda lst: list({t for sub in lst for t in sub}))
#          .reset_index(name="tags")
#     )
#     rolled = rolled.merge(rolled_tags, on=["stream_date"])

#     # derived metrics
#     rolled["subs_per_avg_viewer"]  = rolled["total_subscriptions"]   / rolled["avg_concurrent_viewers"]
#     rolled["chat_msgs_per_viewer"] = rolled["total_num_chats"]       / rolled["avg_concurrent_viewers"]
#     rolled["gift_subs_bool"]       = rolled["gifted_subs_received"]  > 0
#     rolled["created_at"]           = pd.Timestamp.now()

#     # final column order
#     final_cols = [
#         "stream_name","total_subscriptions","stream_date","day_of_week","is_weekend","is_holiday",
#         "stream_start_time","days_since_previous_stream","stream_duration","avg_concurrent_viewers",
#         "peak_concurrent_viewers","unique_viewers","viewer_growth_rate","total_num_chats","total_chatters",
#         "chat_msgs_per_minute","total_emotes_used","unique_emotes_used","followers_start","followers_end",
#         "net_follower_change","new_subscriptions_t1","new_subscriptions_t2_t3","resubscriptions",
#         "gifted_subs_received","gifted_subs_given","subscription_cancellations","bits_donated",
#         "donation_events_count","total_donation_amount","raids_received","raid_viewers_received",
#         "polls_run","poll_participation","predictions_run","prediction_participants","game_category",
#         "category_changes","title_length","has_giveaway","has_qna","tags","moderation_actions",
#         "messages_deleted","timeouts_bans","avg_sentiment_score","positive_negative_ratio",
#         "subs_per_avg_viewer","chat_msgs_per_viewer","subs_7d_moving_avg","subs_3d_moving_avg",
#         "viewers_3d_moving_avg","day_over_day_peak_change","created_at","gift_subs_bool"
#     ]
#     return rolled[final_cols]

# # ─── Usage ────────────────────────────────────────────────────────────────
# # assume `df` is your full live_stream snapshot table:
# one_day = rollup_single_day(daily_sessions, stream_name='thelegendyagami', target_date="2025-08-01")

# # then append via
# # one_day.to_sql("daily_stats", engine, if_exists="append", index=False, method="multi")


In [65]:
# one_day.tail()

,stream_name,total_subscriptions,stream_date,day_of_week,is_weekend,is_holiday,stream_start_time,days_since_previous_stream,stream_duration,avg_concurrent_viewers,peak_concurrent_viewers,unique_viewers,viewer_growth_rate,total_num_chats,total_chatters,chat_msgs_per_minute,total_emotes_used,unique_emotes_used,followers_start,followers_end,net_follower_change,new_subscriptions_t1,new_subscriptions_t2_t3,resubscriptions,gifted_subs_received,gifted_subs_given,subscription_cancellations,bits_donated,donation_events_count,total_donation_amount,raids_received,raid_viewers_received,polls_run,poll_participation,predictions_run,prediction_participants,game_category,category_changes,title_length,has_giveaway,has_qna,tags,moderation_actions,messages_deleted,timeouts_bans,avg_sentiment_score,positive_negative_ratio,subs_per_avg_viewer,chat_msgs_per_viewer,subs_7d_moving_avg,subs_3d_moving_avg,viewers_3d_moving_avg,day_over_day_peak_change,created_at,gift_subs_bool
0,thelegendyagami,132,2025-08-01,Friday,False,False,07:53:22.488564,1,5,12.716079,27,15,0.117647,2631,675,3.248148,0,0,None,None,0.0,0,0,1,131,0,0,2147,9,0.0,2,12,0,0,0,0,Balatro,1,7,False,False,"[Veteran, WrestlingFan, AIArt, Variety, AutoTr...",0,0,0,0.600617,None,10.380559,206.903405,None,None,None,None,2025-08-02 13:12:09.127508,True


In [ ]:
# # one_day = rollup_single_day(df, "2025-08-01")
# import json
# from sqlalchemy.dialects.postgresql import JSONB

# if "tags" in daily_sessions.columns:
#     # 1) Ensure every value is JSON-serialisable (lists or None)
#     one_day["tags"] = one_day["tags"].apply(
#         lambda x: x if isinstance(x, (list, dict)) else ([] if pd.isna(x) else [str(x)])
#     )
#     # 2) Serialise to JSON text so psycopg2 no longer treats it as a PG array
#     one_day["tags"] = one_day["tags"].apply(json.dumps)

# dtype_map = {"tags": JSONB()}

# with engine.begin() as conn:
#     one_day.to_sql(
#         "daily_stats",
#         conn,
#         if_exists="append",
#         index=False,
#         method="multi",
#         dtype=dtype_map
#     )


In [ ]:


# # ── 2. EXTRACT snapshots for thelegendyagami ────────────────────────────────
# query = """
# SELECT *
# FROM live_stream
# WHERE stream_name = %(sn)s
# ORDER BY stream_date, stream_start_time, snapshot_time;
# """
# df = pd.read_sql(query, con=engine, params={"sn": "thelegendyagami"})
# df["snapshot_time"] = pd.to_datetime(df["snapshot_time"])  # ensure tz-aware if needed

# # ── 3. TRANSFORM → one row per day-session ──────────────────────────────────
# def keep_last_snapshot(group: pd.DataFrame) -> pd.Series:
#     """
#     Takes the *latest* 5-min row in a session.  
#     That final row already contains the max stream_duration,
#     peak_concurrent_viewers, total_subscriptions, etc.
#     """
#     return group.sort_values("snapshot_time").iloc[-1]

# daily_sessions = (
#     df.groupby(["stream_date", "stream_start_time"], as_index=False)
#       .apply(keep_last_snapshot)
#       .reset_index(drop=True)
# )

# # Drop live_stream’s PK so Postgres autogenerates a fresh one in daily_stats
# daily_sessions = daily_sessions.drop(columns=["snapshot_time", 'id'])
# # print(daily_sessions.tail())

# import json
# from sqlalchemy.dialects.postgresql import JSONB

# # ── After you build daily_sessions but before to_sql ─────────────
# if "tags" in daily_sessions.columns:
#     # 1) Ensure every value is JSON-serialisable (lists or None)
#     daily_sessions["tags"] = daily_sessions["tags"].apply(
#         lambda x: x if isinstance(x, (list, dict)) else ([] if pd.isna(x) else [str(x)])
#     )
#     # 2) Serialise to JSON text so psycopg2 no longer treats it as a PG array
#     daily_sessions["tags"] = daily_sessions["tags"].apply(json.dumps)


# # # 3) Map the column to JSONB so SQLAlchemy casts correctly
# # dtype_map = {"tags": JSONB()}

# # with engine.begin() as conn:
# #     daily_sessions.to_sql(
# #         "daily_stats",
# #         conn,
# #         if_exists="append",
# #         index=False,
# #         method="multi",
# #         dtype=dtype_map          # <── key line
# #     )

# # print(f"{len(daily_sessions)} day-sessions inserted into daily_stats.")


/var/folders/q9/_p8_sdjd2yj2f7k9bwl3lwwh0000gn/T/ipykernel_65306/3120995841.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(["stream_date", "stream_start_time"], as_index=False)


,stream_name,total_subscriptions,stream_date,day_of_week,is_weekend,is_holiday,stream_start_time,days_since_previous_stream,stream_duration,avg_concurrent_viewers,peak_concurrent_viewers,unique_viewers,viewer_growth_rate,total_num_chats,total_chatters,chat_msgs_per_minute,total_emotes_used,unique_emotes_used,followers_start,followers_end,net_follower_change,new_subscriptions_t1,new_subscriptions_t2_t3,resubscriptions,gifted_subs_received,gifted_subs_given,subscription_cancellations,bits_donated,donation_events_count,total_donation_amount,raids_received,raid_viewers_received,polls_run,poll_participation,predictions_run,prediction_participants,game_category,category_changes,title_length,has_giveaway,has_qna,tags,moderation_actions,messages_deleted,timeouts_bans,avg_sentiment_score,positive_negative_ratio,subs_per_avg_viewer,chat_msgs_per_viewer,subs_7d_moving_avg,subs_3d_moving_avg,viewers_3d_moving_avg,day_over_day_peak_change,created_at,gift_subs_bool
19,thelegendyagami,0,2025-07-30,Wednesday,False,False,14:28:20.963720,0,5,7.066667,9,3,0.125000,5,3,1.0,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,ELDEN RING,0,0,False,False,"[""Veteran"", ""AIArt"", ""English"", ""HardestDiffic...",0,0,0,0.65,None,0.0,1.666667,None,None,None,None,None,False
20,thelegendyagami,0,2025-07-31,Thursday,False,False,14:52:19.354120,0,5,10.000000,11,3,0.100000,16,3,3.2,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,ELDEN RING,0,0,False,False,"[""Veteran"", ""AIArt"", ""English"", ""HardestDiffic...",0,0,0,0.75,None,0.0,5.333333,None,None,None,None,None,True
21,thelegendyagami,0,2025-07-31,Thursday,False,False,19:28:16.030904,0,5,11.866667,13,6,0.181818,18,6,3.6,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Street Fighter 6,0,0,False,False,"[""Veteran"", ""AIArt"", ""English"", ""HardestDiffic...",0,0,0,0.75,None,0.0,3.000000,None,None,None,None,None,False
22,thelegendyagami,0,2025-08-01,Friday,False,False,07:53:22.488564,0,5,17.133333,19,13,0.187500,42,13,8.4,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Jackbox Party Packs,0,0,False,False,"[""Veteran"", ""AIArt"", ""English"", ""HardestDiffic...",0,0,0,0.85,None,0.0,3.230769,None,None,None,None,None,True
23,thelegendyagami,0,2025-08-01,Friday,False,False,22:56:57.339489,0,5,17.800000,19,8,0.117647,40,8,8.0,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Jackbox Party Packs,0,0,False,False,"[""Veteran"", ""AIArt"", ""English"", ""HardestDiffic...",0,0,0,0.75,None,0.0,5.000000,None,None,None,None,None,True


,stream_name,total_subscriptions,stream_date,day_of_week,is_weekend,is_holiday,stream_start_time,days_since_previous_stream,stream_duration,avg_concurrent_viewers,peak_concurrent_viewers,unique_viewers,viewer_growth_rate,total_num_chats,total_chatters,chat_msgs_per_minute,total_emotes_used,unique_emotes_used,followers_start,followers_end,net_follower_change,new_subscriptions_t1,new_subscriptions_t2_t3,resubscriptions,gifted_subs_received,gifted_subs_given,subscription_cancellations,bits_donated,donation_events_count,total_donation_amount,raids_received,raid_viewers_received,polls_run,poll_participation,predictions_run,prediction_participants,game_category,category_changes,title_length,has_giveaway,has_qna,tags,moderation_actions,messages_deleted,timeouts_bans,avg_sentiment_score,positive_negative_ratio,subs_per_avg_viewer,chat_msgs_per_viewer,subs_7d_moving_avg,subs_3d_moving_avg,viewers_3d_moving_avg,day_over_day_peak_change,created_at,gift_subs_bool,min_sentiment_score,max_sentiment_score
188,luthimir,0,2025-08-01,Friday,False,False,22:56:56.660766,2,106,101.048077,127,23,0.000000,281,23,2.666544,33,33,2938.0,2938.0,0,0,0,0,0,0,0,0,0,0.0,1,5,0,0,0,0,Valheim,1,7,False,False,"[HedgeMagic, Chill, SoundOClock, English, Just...",0,0,0,0.750000,None,0.000000,12.217391,4.750000,2.000000,None,51.0,NaT,False,0.55,0.85
189,peach,19,2025-08-01,Friday,False,False,22:56:55.991473,0,407,314.244444,346,187,0.209790,4112,187,10.113175,200,200,22716.0,22745.0,29,2,0,5,12,0,0,200,1,0.0,2,9,0,0,0,0,Just Chatting,2,13,False,False,"[English, Vtuber, ENVtuber, ADHD, Chatty, Loud...",0,0,0,0.850000,None,0.060462,21.978610,27.666667,0.000000,None,243.0,NaT,True,0.45,0.85
190,thelegendyagami,132,2025-08-01,Friday,False,False,07:53:22.488564,1,5,12.716079,27,15,0.117647,2631,675,3.248148,0,0,NaN,NaN,0,0,0,1,131,0,0,2147,9,0.0,2,12,0,0,0,0,Balatro,1,7,False,False,"[""Veteran"", ""WrestlingFan"", ""AIArt"", ""Variety""...",0,0,0,0.600617,None,10.380559,206.903405,NaN,NaN,None,NaN,2025-08-02 13:12:09.127508,True,NaN,NaN
191,alinity,27,2025-08-02,Saturday,True,False,10:02:21.290537,1,251,2526.964143,3849,842,97.692308,8358,842,33.209553,371,371,1524256.0,1524256.0,0,4,0,9,14,0,0,850,7,0.0,0,0,0,0,0,0,Just Chatting,1,13,False,False,"[English, Latina, Woman, girl, Colombian, Cana...",0,0,0,0.850000,None,0.010685,9.861045,41.500000,36.666667,None,2595.0,NaT,True,0.45,0.85
192,ellie_minibot,185,2025-08-02,Saturday,True,False,13:19:21.963763,4,133,841.804511,1216,456,45.769231,3035,456,22.992352,211,211,60729.0,60729.0,0,6,0,42,137,0,0,501,2,0.0,2,2,0,0,0,0,Just Chatting,1,13,False,False,"[vtuber, english, goofy, chill, cozy, educatio...",0,0,0,0.850000,None,0.219766,6.655702,210.000000,0.000000,None,-278.0,NaT,True,0.30,0.95


In [27]:
df_daily['start_time_hour'] = (
    pd.to_datetime(df_daily['stream_start_time'].astype(str))
      .dt.hour
)

# 3) Filter to “late night” → between 23:00 and 04:00
mask = (df_daily['start_time_hour'] >= 23) | (df_daily['start_time_hour'] < 12)
night_df = df_daily[mask]

# 4) Peek at the first 15 rows, all columns
night_df.tail()

/var/folders/q9/_p8_sdjd2yj2f7k9bwl3lwwh0000gn/T/ipykernel_65306/3317650635.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df_daily['stream_start_time'].astype(str))


,stream_name,total_subscriptions,stream_date,day_of_week,is_weekend,is_holiday,stream_start_time,days_since_previous_stream,stream_duration,avg_concurrent_viewers,peak_concurrent_viewers,unique_viewers,viewer_growth_rate,total_num_chats,total_chatters,chat_msgs_per_minute,total_emotes_used,unique_emotes_used,followers_start,followers_end,net_follower_change,new_subscriptions_t1,new_subscriptions_t2_t3,resubscriptions,gifted_subs_received,gifted_subs_given,subscription_cancellations,bits_donated,donation_events_count,total_donation_amount,raids_received,raid_viewers_received,polls_run,poll_participation,predictions_run,prediction_participants,game_category,category_changes,title_length,has_giveaway,has_qna,tags,moderation_actions,messages_deleted,timeouts_bans,avg_sentiment_score,positive_negative_ratio,subs_per_avg_viewer,chat_msgs_per_viewer,subs_7d_moving_avg,subs_3d_moving_avg,viewers_3d_moving_avg,day_over_day_peak_change,created_at,gift_subs_bool,min_sentiment_score,max_sentiment_score,start_time_hour
166,raysfire,4,2025-07-31,Thursday,False,False,01:28:16.474332,1,229,252.143478,341,89,0.175862,1587,89,6.930096,141,141,99091,99093,2,0,0,4,0,0,0,8,4,0.0,2,260,0,0,0,0,Donkey Kong Bananza,1,19,False,False,"[WINNABLE, FOX, English, GRINDEL, NoEggResidue...",0,0,0,0.75,None,0.015864,17.831461,3.000000,2.000000,None,-171.0,None,False,0.45,0.85,1
167,barbarousking,171,2025-07-31,Thursday,False,False,08:02:17.505562,2,454,3990.165198,5431,2766,26.019900,22043,2766,48.656864,681,681,229706,229706,0,1,0,110,60,0,0,48574,49,0.0,0,0,0,0,0,0,Street Fighter 6,4,10,False,False,"[BaldWelcome, Kaizo, HardMode, GIGACHAD, Engli...",0,0,0,0.85,None,0.042855,7.969270,109.833333,140.333333,None,1397.0,None,True,0.20,0.85,8
169,boesschen,32,2025-07-31,Thursday,False,False,08:09:17.807158,1,565,140.509735,195,145,47.750000,6577,145,11.642069,349,349,7834,7834,0,1,0,5,26,0,0,17592,47,0.0,2,46,0,0,0,0,Marbles on Stream,2,13,False,False,"[Vtuber, ENVtuber, English, Chatty, Playingwit...",0,0,0,0.85,None,0.227742,45.358621,25.333333,19.000000,None,19.0,None,True,0.45,0.95,8
179,barbarousking,171,2025-08-01,Friday,False,False,08:11:22.733255,1,474,3502.322650,4257,2299,10.028497,17515,2299,36.918093,613,613,229718,229718,0,3,0,111,57,0,0,21000,30,0.0,0,0,0,0,0,0,Titan Quest II,5,23,False,False,"[BaldWelcome, Kaizo, HardMode, GIGACHAD, Engli...",0,0,0,0.85,None,0.048825,7.610265,124.666667,148.500000,None,1808.0,None,True,0.25,0.85,8
183,necros,154,2025-08-01,Friday,False,False,08:52:24.621294,2,782,6072.500649,12232,5973,348.485714,52463,5973,67.080047,305,305,907340,907340,0,53,0,63,38,0,0,6600,8,0.0,2,149,0,0,0,0,Marvel Rivals,1,13,False,False,"[English, spiderman, blackpanther, DropsEnabled]",0,0,0,0.45,None,0.025360,8.781946,35.250000,39.500000,None,5088.0,None,True,0.20,0.95,8


In [18]:
# Example: pull every row from your 'users' table
# show all columns and let pandas auto-width to your terminal/notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# # now read and print
# df = pd.read_sql(
#     "SELECT * FROM live_stream WHERE total_subscriptions > 0;", 
#     con=engine)

df_live = pd.read_sql(
    "SELECT * FROM live_stream;", 
    con=engine)

cols = ['stream_name'] + ['total_subscriptions'] + [c for c in df_live.columns if c != 'stream_name' and c != 'total_subscriptions' and c != 'id']
df_live = df_live[cols]

# display
df_live.tail(35)


,stream_name,total_subscriptions,snapshot_time,stream_date,day_of_week,is_weekend,is_holiday,stream_start_time,days_since_previous_stream,stream_duration,avg_concurrent_viewers,peak_concurrent_viewers,unique_viewers,viewer_growth_rate,total_num_chats,total_chatters,chat_msgs_per_minute,total_emotes_used,unique_emotes_used,followers_start,followers_end,net_follower_change,new_subscriptions_t1,new_subscriptions_t2_t3,resubscriptions,gifted_subs_received,gifted_subs_given,subscription_cancellations,bits_donated,donation_events_count,total_donation_amount,raids_received,raid_viewers_received,polls_run,poll_participation,predictions_run,prediction_participants,game_category,category_changes,title_length,has_giveaway,has_qna,tags,moderation_actions,messages_deleted,timeouts_bans,avg_sentiment_score,positive_negative_ratio,subs_per_avg_viewer,chat_msgs_per_viewer,subs_7d_moving_avg,subs_3d_moving_avg,viewers_3d_moving_avg,day_over_day_peak_change,created_at,gift_subs_bool
25578,alinity,0,2025-08-19 17:16:10.630164,2025-08-19,Tuesday,False,False,12:03:07.348608,0,5,1258.200000,1402,27,0.000000,107,27,21.4,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Mafia: The Old Country,0,0,False,False,"[English, Latina, Woman, girl, Colombian, Cana...",0,0,0,0.35,None,0.000000,3.962963,None,None,None,None,None,True
25579,boesschen,0,2025-08-19 17:16:11.227462,2025-08-19,Tuesday,False,False,08:07:05.682639,0,5,281.933333,312,14,0.043478,39,14,7.8,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Marvel Rivals,0,0,False,False,"[Vtuber, ENVtuber, English, Chatty, Playingwit...",0,0,0,0.65,None,0.000000,2.785714,None,None,None,None,None,True
25580,strikerhutassa,0,2025-08-19 17:17:07.486616,2025-08-19,Tuesday,False,False,14:45:06.055172,0,5,10.200000,11,1,0.000000,3,1,0.6,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Destiny 2,0,0,False,False,"[English, chill, Positivity, LGBTQFriendly, Pl...",0,0,0,0.45,None,0.000000,3.000000,None,None,None,None,None,False
25581,jynxzi,19,2025-08-19 17:22:06.967504,2025-08-19,Tuesday,False,False,15:53:06.278480,0,5,39684.866667,42218,954,0.043528,2012,954,402.4,0,0,None,None,0.0,10,0,9,0,0,0,0,0,0.0,0,0,0,0,0,0,Clash Royale,0,0,False,False,[English],0,0,0,0.45,None,0.000479,2.109015,None,None,None,None,None,True
25582,shroud,1,2025-08-19 17:22:08.470723,2025-08-19,Tuesday,False,False,13:40:05.719878,0,5,8434.666667,9419,124,0.000000,199,124,39.8,0,0,None,None,0.0,0,0,1,0,0,0,0,0,0.0,0,0,0,0,0,0,Battlefield 2042,1,0,False,False,"[English, DropsEnabled]",0,0,0,0.65,None,0.000119,1.604839,None,None,None,None,None,False
25583,smallant,0,2025-08-19 17:22:09.107785,2025-08-19,Tuesday,False,False,15:04:06.901551,0,5,2001.533333,2017,52,0.004983,98,52,19.6,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Donkey Kong Bananza,0,0,False,False,[English],0,0,0,0.75,None,0.000000,1.884615,None,None,None,None,None,False
25584,alinity,0,2025-08-19 17:22:10.050665,2025-08-19,Tuesday,False,False,12:03:07.348608,0,5,1314.333333,1339,25,0.021358,75,25,15.0,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Mafia: The Old Country,0,0,False,False,"[English, Latina, Woman, girl, Colombian, Cana...",0,0,0,0.45,None,0.000000,3.000000,None,None,None,None,None,True
25585,boesschen,0,2025-08-19 17:22:10.692215,2025-08-19,Tuesday,False,False,08:07:05.682639,0,5,276.533333,308,13,0.166667,40,13,8.0,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Marvel Rivals,0,0,False,False,"[Vtuber, ENVtuber, English, Chatty, Playingwit...",0,0,0,0.65,None,0.000000,3.076923,None,None,None,None,None,True
25586,strikerhutassa,0,2025-08-19 17:22:11.635005,2025-08-19,Tuesday,False,False,14:45:06.055172,0,5,10.400000,12,3,0.200000,9,3,1.8,0,0,None,None,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,Destiny 2,0,0,False,False,"[English, chill, Positivity, LGBTQFriendly, Pl...",0,0,0,0.65,None,0.000000,3.000000,None,None,None,None,None,False
25587,shroud,0,2025-08-19 17:28:07.877857,2025-08-19,Tuesday,False,False,13:40:05.719878,0,5,8605.266667,10166,160,0.136374,253,16

In [ ]:

# schema_query = """
# SELECT column_name,
#        data_type
# FROM   information_schema.columns
# WHERE  table_name = 'live_stream'
#   AND  table_schema = 'public';   -- optional, if your table lives in public schema
# """
# df_schema = pd.read_sql(schema_query, con=engine)
# pd.set_option('display.max_rows', None)
# print(df_schema)

In [ ]:
# from sqlalchemy import text

# with engine.begin() as conn:
#     conn.execute(text("""
#         ALTER TABLE daily_stats
#         DROP COLUMN snapshot_time;
#     """))


In [ ]:
# from sqlalchemy import text

# with engine.begin() as conn:
#     conn.execute(text("""
#         /* ─────── primary key ──────────────────────────────────── */

#         -- id: bigint → integer (because your model uses db.Integer)
#         ALTER TABLE live_stream
#           ALTER COLUMN id
#           TYPE integer
#           USING id::integer;

#         /* ─────── core scalars ───────────────────────────────────── */

#         -- stream_name: text → varchar(128)
#         ALTER TABLE live_stream
#           ALTER COLUMN stream_name
#           TYPE varchar(128)
#           USING stream_name::varchar;

#         /* ─────── dates & times ─────────────────────────────────── */

#         -- YYYY-MM-DD
#         ALTER TABLE live_stream
#           ALTER COLUMN stream_date
#           TYPE date
#           USING stream_date::date;

#         -- HH:MM:SS
#         ALTER TABLE live_stream
#           ALTER COLUMN stream_start_time
#           TYPE time
#           USING stream_start_time::time;

#         /* ─────── small varchar fields ──────────────────────────── */

#         -- Monday … Sunday
#         ALTER TABLE live_stream
#           ALTER COLUMN day_of_week
#           TYPE varchar(9)
#           USING day_of_week::varchar;

#         -- Game / category name
#         ALTER TABLE live_stream
#           ALTER COLUMN game_category
#           TYPE varchar(128)
#           USING game_category::varchar;

#         /* ─────── text → integer for followers & net change ─────── */

#         ALTER TABLE live_stream
#           ALTER COLUMN followers_start
#           TYPE integer
#           USING followers_start::integer,
#           ALTER COLUMN followers_end
#           TYPE integer
#           USING followers_end::integer,
#           ALTER COLUMN net_follower_change
#           TYPE integer
#           USING net_follower_change::integer;

#         /* ─────── text → double precision for moving averages ──── */

#         ALTER TABLE live_stream
#           ALTER COLUMN subs_7d_moving_avg
#           TYPE double precision
#           USING subs_7d_moving_avg::double precision,
#           ALTER COLUMN subs_3d_moving_avg
#           TYPE double precision
#           USING subs_3d_moving_avg::double precision,
#           ALTER COLUMN day_over_day_peak_change
#           TYPE double precision
#           USING day_over_day_peak_change::double precision;

#         /* ─────── JSON → jsonb ─────────────────────────────────── */

#         ALTER TABLE live_stream
#           ALTER COLUMN tags
#           TYPE jsonb
#           USING tags::jsonb;

#         /* ─────── float conversions (double precision) ─────────── */

#         ALTER TABLE live_stream
#           ALTER COLUMN positive_negative_ratio
#           TYPE double precision
#           USING positive_negative_ratio::double precision,
#           ALTER COLUMN viewers_3d_moving_avg
#           TYPE double precision
#           USING viewers_3d_moving_avg::double precision;

#         /* ─────── timestamp field ───────────────────────────────── */

#         ALTER TABLE live_stream
#           ALTER COLUMN created_at
#           TYPE timestamp without time zone
#           USING created_at::timestamp;

#         /* ─── all other columns already match:
#              – bigint ↔︎ Integer (after id change)
#              – double precision ↔︎ Float
#              – boolean ↔︎ Boolean
#         */
#     """))


# Dummy Bot

In [ ]:
import nest_asyncio, asyncio, random, time, itertools, pprint, types
nest_asyncio.apply()            # lets us run `await` in cells

# 1)  Build a throw‑away Flask app
from flask import Flask
from db import db                # ← the SINGLETON used by your models

In [ ]:
app = Flask(__name__)
app.config.update(
    SQLALCHEMY_DATABASE_URI="sqlite:///:memory:",   # in‑memory for tests
    SQLALCHEMY_TRACK_MODIFICATIONS=False,
)

db.init_app(app)                 # bind THAT instance to THIS app

with app.app_context():
    from models import DailyStats    # safe to import now
    db.create_all()


In [ ]:
from twitchio.ext import commands
from types import SimpleNamespace

class DummyBot(commands.Bot):
    def __init__(self):
        super().__init__(
            token='dummy',            # never used
            nick='simulated_channel',
            prefix='!',
            initial_channels=[]
        )
        self._ready = asyncio.Event()
        self._streams_iter = self._stream_generator()

    async def wait_for_ready(self):
        await self._ready.wait()

    async def fetch_users(self, names=None, **kw):
        return [SimpleNamespace(id=12345, followers=1500)]

    async def fetch_streams(self, user_ids=None, user=None):
        return next(self._streams_iter)

    # --- simulation helpers ---
    def _stream_generator(self):
        # 2 ticks offline, 10 ticks live, then offline
        for _ in range(2): yield []
        for v in range(50, 151, 10):
            yield [SimpleNamespace(viewer_count=v)]
        while True: yield []

    async def _ws_connect(self, *a, **kw):
        """Override real websocket connection (never called here)."""
        raise RuntimeError("Dummy bot doesn't connect")


In [ ]:
from cogs.daily_stats_collector import DailyStatsCollector

bot = DummyBot()
collector = DailyStatsCollector(bot)          # __init__ already did .start()

# shorten the tick to 1 s
collector.metrics_collector.change_interval(seconds=1)

from unittest.mock import AsyncMock
collector._on_stream_end = AsyncMock()   # monkey‑patch



In [ ]:
# bot = DummyBot()
# collector = DailyStatsCollector(bot)

# cancel 60‑s default, switch to 1‑s
collector.metrics_collector.cancel()
collector.metrics_collector.change_interval(seconds=1)
collector.metrics_collector.start()


In [ ]:
async def run_sim():
    bot._ready.set()
    await asyncio.sleep(15)
    if collector.stream_live:
        await collector._on_stream_end()

    pprint.pp(collector.current_stats)     # stats before DB insert

with app.app_context():                    # <‑‑ context solves RuntimeError
    await run_sim()

# query what was written
with app.app_context():
    row = DailyStats.query.first()
    if row:
        print("peak viewers ->", row.peak_concurrent_viewers)
    else:
        print("No rows written — check that _on_stream_end ran and wasn't mocked.")



# Delete table!!!

In [ ]:
with app.app_context():                      # ensure a Flask context
    from models import DailyStats

    # DELETE FROM daily_stats;
    DailyStats.query.delete(synchronize_session=False)
    db.session.commit()

    # optional sanity check
    print("Rows left →", DailyStats.query.count())


In [ ]:
import random
import numpy as np
import math
import datetime as dt
from collections import deque
from typing import List

In [ ]:

PROB_BASE_GIFT = 0.02   # base gift-sub probability
# logistic weights: tune these to make gift subs more or less sensitive
W_CHAT_RATE = 3.0
W_SUB_RATE  = 2.0
W_GIFT_MA   = 1.5

def sigmoid(x: float) -> float:
    return 1.0 / (1.0 + math.exp(-x))

def make_dummy_row(
    stream_date: dt.date,
    history: List["DailyStats"]
) -> "DailyStats":
    """
    Build one day's stats, using `history` to compute moving averages
    and to drive the gift-sub probability.
    """
    # --- base viewer/chat/sub values ---
    peak = random.randint(80, 200)
    avg  = round(peak * random.uniform(0.4, 0.7), 1)
    total_chats = random.randint(int(avg * 5), int(avg * 20))
    # subs_t1 = random.randint(3, 25)
    stream_duration = random.randint(120, 300)

    # approximate chat‐rate (msgs/min)
    chat_rate = total_chats / stream_duration

    # ----- correlated subscriptions -----
    # base λ for Tier-1 subs scales with avg viewers and chat activity
    lam_subs = avg * 0.005   + chat_rate * 0.1
    lam_subs = max(0.1, lam_subs)   # floor so you still get occasional subs
    subs_t1 = max(0, np.random.poisson(lam_subs))


    # ----- correlated bits -----
    # assume typical bits donated per chat ~0.5, plus a small boost per viewer
    lam_bits = total_chats * 0.5 + avg * 2
    bits_donated = max(0, int(round(random.gauss(lam_bits, lam_bits * 0.3))))

    if history:
        days_since = (stream_date - history[-1].stream_date).days
    else:
        days_since = 1

    # --- time-series features from history ---
    last3 = history[-3:]
    viewers_3d_moving_avg = (
        round(sum(d.peak_concurrent_viewers for d in last3) / len(last3), 1)
        if len(last3) >= 3 else None
    )
    last7 = history[-7:]
    subs_7d_moving_avg = (
        round(sum(d.gifted_subs_received for d in last7) / len(last7), 2)
        if len(last7) >= 7 else None
    )
    day_over_day_peak_change = (
        round((peak - history[-1].peak_concurrent_viewers) 
              / history[-1].peak_concurrent_viewers, 2)
        if history else 0
    )

    # --- logistic model for gift-sub probability ---
    sub_rate  = subs_t1 / (avg or 1)
    gift_ma   = subs_7d_moving_avg or 0

    logit = (W_CHAT_RATE * (chat_rate - 1.5)
             + W_SUB_RATE  * (sub_rate  - 0.1)
             + W_GIFT_MA   * (gift_ma   - 1))
    
    p_gift = sigmoid(logit) * (1 - PROB_BASE_GIFT) + PROB_BASE_GIFT

    if subs_t1 > 0 and random.random() < p_gift:
        max_gift = min(15, subs_t1)
        gifted = random.randint(1, max_gift)
    else:
        gifted = 0

    
    delta = int(round(random.gauss(0, 1)))

    # 2) clamp to ±3 so you never get huge jumps
    delta = max(0, min(4, delta))
    followers_start         = 1500 + (1 * len(history))
    gifted_bool = True if int(gifted > 0) else False

    # --- assemble the DailyStats record ---
    return DailyStats(
        stream_date             = stream_date,
        day_of_week             = stream_date.strftime("%A"),
        is_weekend              = stream_date.weekday() >= 5,
        is_holiday              = False,
        stream_start_time       = dt.time(hour=random.randint(18, 22), minute=0),
        days_since_previous_stream = days_since,
        stream_duration         = stream_duration,
        avg_concurrent_viewers  = avg,
        peak_concurrent_viewers = peak,
        unique_viewers          = int(avg * 2),
        viewer_growth_rate      = round((peak - avg) / (avg or 1), 2),
        total_num_chats         = total_chats,
        total_chatters          = int(total_chats / random.randint(8, 15)),
        chat_msgs_per_minute    = round(total_chats / (stream_duration or 1), 2),
        total_emotes_used       = random.randint(100, 800),
        unique_emotes_used      = random.randint(20, 80),
        followers_start         = followers_start,
        followers_end           = followers_start + delta,
        net_follower_change     = delta,  # you can recompute if desired
        total_subscriptions     = subs_t1 + gifted,
        new_subscriptions_t1    = subs_t1,
        new_subscriptions_t2_t3 = random.randint(0, 3),
        resubscriptions         = random.randint(0, 10),
        gifted_subs_received    = gifted,
        gifted_subs_given       = 0,
        subscription_cancellations = random.randint(0, 2),
        bits_donated            = bits_donated,
        donation_events_count   = random.randint(0, 15),
        total_donation_amount   = round(random.uniform(0, 80), 2),
        raids_received          = random.randint(0, 3),
        raid_viewers_received   = random.randint(0, 300),
        polls_run               = random.randint(0, 4),
        poll_participation      = random.randint(0, 150),
        predictions_run         = random.randint(0, 3),
        prediction_participants = random.randint(0, 100),
        game_category           = random.choice(["Just Chatting", "Fortnite", "Minecraft"]),
        category_changes        = random.randint(0, 2),
        title_length            = random.randint(20, 60),
        has_giveaway            = random.choice([True, False]),
        has_qna                 = random.choice([True, False]),
        tags                    = random.choice(["AI", "Testing"]),
        moderation_actions      = random.randint(0, 10),
        messages_deleted        = random.randint(0, 5),
        timeouts_bans           = random.randint(0, 3),
        avg_sentiment_score     = round(random.uniform(-1, 1), 2),
        positive_negative_ratio = round(random.uniform(0.5, 4.0), 2),
        subs_per_avg_viewer     = round((subs_t1 + gifted) / (avg or 1), 2),
        chat_msgs_per_viewer    = round(total_chats / (avg * 2 or 1), 2),
        subs_7d_moving_avg      = subs_7d_moving_avg,
        viewers_3d_moving_avg   = viewers_3d_moving_avg,
        day_over_day_peak_change= day_over_day_peak_change,
        gift_subs_bool          = gifted_bool,
    )

def generate_synthetic_data(
    num_streams: int = 60,
    skip_prob: float = 0.1,
    always_skip_mondays: bool = True
) -> List["DailyStats"]:
    """
    Return `num_streams` DailyStats rows, on realistic calendar dates:
    - skips Mondays entirely (if always_skip_mondays=True)
    - on other days, skips with probability skip_prob
    """
    history = []
    # start far enough back that skipping still gives you ~num_streams
    current = dt.date.today() - dt.timedelta(days=num_streams * 2)

    while len(history) < num_streams:
        # 1) decide whether to skip this date
        if always_skip_mondays and current.weekday() == 0:
            current += dt.timedelta(days=1)
            continue
        if random.random() < skip_prob:
            current += dt.timedelta(days=1)
            continue

        # 2) generate a stream on `current`
        row = make_dummy_row(current, history)
        history.append(row)

        # 3) move to next calendar day
        current += dt.timedelta(days=1)

    return history

In [ ]:
N_STREAMS = 1000      # how many synthetic streams you want
SKIP_PROB  = 0.1    # 10% random off-days
MONDAY_SKIP = True  # always skip Mondays

with app.app_context():
    # 1) delete old synthetic data
    DailyStats.query.delete()
    db.session.commit()

    # 2) generate new synthetic streams
    rows = generate_synthetic_data(
        num_streams=N_STREAMS,
        skip_prob=SKIP_PROB,
        always_skip_mondays=MONDAY_SKIP
    )

    # 3) bulk‐insert
    db.session.bulk_save_objects(rows)
    db.session.commit()

    print(f"Inserted {len(rows)} synthetic DailyStats rows.")


In [ ]:
# 1. Allow pandas to show *all* columns
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)           # or any suitably large number
pd.set_option("display.max_colwidth", None)

# 2. Load your table and show the first 10 rows
with app.app_context():
    df = pd.read_sql_table("daily_stats", db.engine)

print(df.head(10))

In [ ]:
df = df.copy()
df.columns = df.columns.map(str)      # cast each label to a real Python str


In [ ]:
# 1. Sort by date so shifting makes sense
df = df.sort_values("stream_date")

# 2. Create your “next-day” target
df["gift_subs_tomorrow"] = df["gift_subs_bool"].shift(-1)

# 3. Drop the final row (it has NaN for the shifted target)
df = df.dropna(subset=["gift_subs_tomorrow"])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 3) Show summary statistics
pd.set_option('display.max_columns', None)
print("=== Summary Statistics ===")
print(df.describe().T)

# 4) Missing Values Heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False)
plt.title('Missing Values Heatmap')
plt.show()

# 5) Numeric Feature Correlation Matrix
plt.figure(figsize=(24, 20))
numeric_df = df.select_dtypes(include=[np.number])
corr = numeric_df.corr()
sns.heatmap(corr, annot=True, fmt='.2f', square=True)
plt.title('Numeric Feature Correlation Matrix')
plt.show()

# 6) Distributions of Key Features
key_features = [
    'avg_concurrent_viewers',
    'total_num_chats',
    'chat_msgs_per_minute',
    'new_subscriptions_t1',
    'gifted_subs_received',
    'bits_donated'
]
for col in key_features:
    plt.figure()
    sns.histplot(df[col].dropna(), bins=30, kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.show()

# 7) Pairplot of Key Features vs. gift_subs_bool
sns.pairplot(df[key_features + ['gift_subs_bool']], hue='gift_subs_bool', corner=True)
plt.suptitle('Pairplot of Key Features Colored by gift_subs_bool', y=1.02)
plt.show()


In [ ]:
# your X (features) and y (target)
y = df["gift_subs_tomorrow"]
X = df.drop(columns=["gift_subs_tomorrow"])

In [ ]:
print(y.value_counts())
print(y.value_counts(normalize=True))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numeric = X.select_dtypes(include="number").columns

for feat in ["chat_msgs_per_minute", "avg_concurrent_viewers", "subs_7d_moving_avg", "bits_donated"]:
    plt.figure(figsize=(6,4))
    sns.boxplot(x=y, y=X[feat])
    plt.title(f"{feat} by gift_subs_tomorrow")
    plt.xlabel("gift_subs_tomorrow")
    plt.ylabel(feat)
    plt.show()



In [ ]:
from scipy.stats import pointbiserialr

corrs = {}
for feat in numeric:
    # ignore NaNs
    valid = df[[feat, "gift_subs_tomorrow"]].dropna()
    r, p = pointbiserialr(valid[feat], valid["gift_subs_tomorrow"])
    corrs[feat] = r

# sort by absolute correlation
sorted(corrs.items(), key=lambda kv: abs(kv[1]), reverse=True)


In [ ]:
cats = X.select_dtypes(include="bool").columns.tolist() + ["game_category"]
for cat in cats:
    plt.figure(figsize=(5,3))
    sns.countplot(x=cat, hue=y, data=df)
    plt.title(f"{cat} vs gift_subs_tomorrow")
    plt.show()


In [ ]:
sns.pairplot(
    df,
    vars=["chat_msgs_per_minute","avg_concurrent_viewers","subs_7d_moving_avg"],
    hue="gift_subs_tomorrow",
    corner=True
)
plt.show()


In [ ]:
# 1) Load & sort
df = df.sort_index()

# 2) Select features & target
FEATURES = [
    "stream_date",
    "day_of_week",
    "is_weekend",
    "is_holiday",
    "stream_start_time",
    "days_since_previous_stream",
    "stream_duration",
    "avg_concurrent_viewers",
    "peak_concurrent_viewers",
    "unique_viewers",
    "viewer_growth_rate",
    "total_num_chats",
    "total_chatters",
    "chat_msgs_per_minute",
    "total_emotes_used",
    "unique_emotes_used",
    "followers_start",
    "followers_end",
    "net_follower_change",
    "total_subscriptions",
    "new_subscriptions_t1",
    "new_subscriptions_t2_t3",
    "resubscriptions",
    "gifted_subs_received",
    "gifted_subs_given",
    "subscription_cancellations",
    "bits_donated",
    "donation_events_count",
    "total_donation_amount",
    "raids_received",
    "raid_viewers_received",
    "polls_run",
    "poll_participation",
    "predictions_run",
    "prediction_participants",
    "game_category",
    "category_changes",
    "title_length",
    "has_giveaway",
    "has_qna",
    "tags",
    "moderation_actions",
    "messages_deleted",
    "timeouts_bans",
    "avg_sentiment_score",
    "positive_negative_ratio",
    "subs_per_avg_viewer",
    "chat_msgs_per_viewer",
    "subs_7d_moving_avg",
    "viewers_3d_moving_avg",
    "day_over_day_peak_change",
    "gift_subs_bool",
]

TARGET = "gift_subs_tomorrow"

print(df.head())


X = df[FEATURES]
y = df[TARGET].values

# 2) TRAIN/TEST SPLIT on the raw data!
split_idx   = int(len(df) * 0.8)
X_train_df  = X.iloc[:split_idx]
y_train     = y[:split_idx]
X_test_df   = X.iloc[split_idx:]
y_test      = y[split_idx:]


# 3) IDENTIFY numeric vs. categorical
numeric_feats = X_train_df.select_dtypes(include=["number"]).columns.tolist()
cat_feats     = X_train_df.select_dtypes(exclude=["number"]).columns.tolist()

X_train = np.array(X_train_df)
print("Shape X_train",X_train.shape)

X_test = np.array(X_test_df)
print("Shape X_test",X_test.shape)

In [ ]:
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale",  StandardScaler())
])
cat_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
    ("encode", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])
preprocessor = ColumnTransformer([
    ("num", num_pipeline, numeric_feats),
    ("cat", cat_pipeline, cat_feats),
])

# 5) FIT on train, TRANSFORM train & test
X_train_proc = preprocessor.fit_transform(X_train_df)
X_test_proc  = preprocessor.transform(X_test_df)

X_train_proc.shape

In [ ]:
X_test_proc.shape

In [ ]:
SEQ_LEN = 7   # use past week to predict tomorrow

def make_sequences(X: np.ndarray, y: np.ndarray, seq_len: int=SEQ_LEN):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i : i + seq_len])
        ys.append(y[i + seq_len])
    return np.stack(Xs), np.array(ys)

X_train_seq, y_train_seq = make_sequences(X_train_proc, y_train, seq_len=SEQ_LEN)
X_test_seq,  y_test_seq  = make_sequences(X_test_proc,  y_test,  seq_len=SEQ_LEN)

val_frac = 0.2

n_train = len(X_train_seq)
split_at = int(n_train * (1 - val_frac))

# First part stays training…
X_train_final = X_train_seq[:split_at]
y_train_final = y_train_seq[:split_at]

# Last 20% becomes validation
X_val        = X_train_seq[split_at:]
y_val        = y_train_seq[split_at:]

In [ ]:
print(X_train_final.shape)
# print(X_train_final[0][4])
print(y_train_final.shape)

In [ ]:
SEQ_LEN, N_FEATURES = X_train_seq.shape[1], X_train_seq.shape[2]

model = models.Sequential([
    layers.Input(shape=(SEQ_LEN, N_FEATURES)),
    
    # Block 1
    layers.Conv1D(64, kernel_size=3, padding='same', 
                  kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Conv1D(64, kernel_size=5, padding='same', 
                  kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.1),
    
    # Block 2
    layers.Conv1D(128, kernel_size=3, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.GlobalAveragePooling1D(),
    
    # Dense head
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid'),
])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy', 'AUC']
)


model.summary()

# 8) Fit with early stopping
es = callbacks.EarlyStopping(
    monitor="val_auc", patience=5, mode="max", restore_best_weights=True
)

In [ ]:
history = model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[es]
)

# 9) Evaluate
val_loss, val_acc, val_auc = model.evaluate(X_val, y_val, verbose=0)
print(f"Val AUC: {val_auc:.3f}")


In [ ]:
# Extract metrics
acc      = history.history['accuracy']
loss     = history.history['loss']
val_acc  = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs   = range(1, len(acc) + 1)

# 1) Training Accuracy & Loss
fig, ax1 = plt.subplots(figsize=(8, 4))
ax1.plot(epochs, acc,     'b-', label='Train Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy', color='b')
ax1.tick_params(axis='y', labelcolor='b')

ax2 = ax1.twinx()
ax2.plot(epochs, loss,    'r-', label='Train Loss')
ax2.set_ylabel('Loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

fig.suptitle('Training Accuracy and Loss')
fig.legend(loc='upper right')
plt.show()

# 2) Validation Accuracy & Loss
fig, ax1 = plt.subplots(figsize=(8, 4))
ax1.plot(epochs, val_acc, 'b--', label='Val Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy', color='b')
ax1.tick_params(axis='y', labelcolor='b')

ax2 = ax1.twinx()
ax2.plot(epochs, val_loss,'r--', label='Val Loss')
ax2.set_ylabel('Loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

fig.suptitle('Validation Accuracy and Loss')
fig.legend(loc='upper right')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 1) Predict and binarize
y_pred_prob = model.predict(X_test_seq)
y_pred      = (y_pred_prob.flatten() >= 0.5).astype(int)

# 2) Compute the confusion matrix
cm = confusion_matrix(y_test_seq, y_pred)

# 3) Plot it
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# 1) Predict on your test sequences
y_pred_prob = model.predict(X_test_seq)
y_pred      = (y_pred_prob.flatten() >= 0.5).astype(int)

# 2) Compute a row-normalized confusion matrix (each row sums to 1)
cm_norm = confusion_matrix(y_test_seq, y_pred, normalize='true')

# 3) Display it with percentages
disp = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=[0, 1])
fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(
    ax=ax,
    cmap=plt.cm.Blues,
    values_format='.2f'  # show as decimals, e.g. 0.85 = 85%
)
plt.title('Normalized Confusion Matrix\n(rows = true class proportions)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
import sys
# Installs the latest scikit-learn into the current environment:
!{sys.executable} -m pip install --upgrade scikit-learn


In [ ]:
# ── 1. Disable metadata routing up front ─────────────────────────────────
from sklearn import set_config, config_context
set_config(enable_metadata_routing=False)                           # no internal metadata lookups :contentReference[oaicite:1]{index=1}

# ── 2. Imports ───────────────────────────────────────────────────────────
from sklearn.model_selection import train_test_split                # splitting data :contentReference[oaicite:2]{index=2}
from sklearn.pipeline        import Pipeline                        # chaining transformers & estimator :contentReference[oaicite:3]{index=3}
from sklearn.compose         import ColumnTransformer               # heterogeneous transforms :contentReference[oaicite:4]{index=4}
from sklearn.impute          import SimpleImputer                   # missing‐value imputation :contentReference[oaicite:5]{index=5}
from sklearn.preprocessing   import StandardScaler, OneHotEncoder    # scaling & one-hot :contentReference[oaicite:6]{index=6}
from sklearn.ensemble        import GradientBoostingRegressor       # regression model :contentReference[oaicite:7]{index=7}
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics         import mean_absolute_error             # evaluation :contentReference[oaicite:8]{index=8}
import numpy as np


In [ ]:
# ── 3. Identify feature types (by name, on the original DataFrame X) ───
target_col = "gift_subs_tomorrow"
X = df.drop(columns=[target_col])
y = df[target_col]

# 4. Build X and y
# feature_cols = [c for c in df.columns
#                 if c not in ("stream_date", "gift_subs_bool", "gift_subs_tomorrow")]
# X = df[feature_cols]
# y = df["gift_subs_tomorrow"].astype(int)


In [ ]:
# ── 7. Create train / val / test splits (60/20/20) ────────────────────
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42
)  # 0.25 * 0.8 = 0.20 of total :contentReference[oaicite:16]{index=16}

In [ ]:
print(len(y_train==1.0))
y_train.head(20)

In [ ]:


numeric_features     = X.select_dtypes(include=["int64","float64"]).columns.tolist()   # e.g. ["avg_concurrent_viewers", …] :contentReference[oaicite:9]{index=9}
categorical_features = X.select_dtypes(include=["object","bool"]    ).columns.tolist()   # e.g. ["has_giveaway", …] :contentReference[oaicite:10]{index=10}

print(numeric_features)
print(categorical_features)

# ── 4. Build the two sub‐pipelines ───────────────────────────────────────
numeric_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),    # numeric median imputation :contentReference[oaicite:11]{index=11}
    ("scale",  StandardScaler()),                     # zero-mean unit-variance :contentReference[oaicite:12]{index=12}
])

categorical_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),        # fill blanks with mode :contentReference[oaicite:13]{index=13}
    ("encode", OneHotEncoder(handle_unknown="ignore")),         # one-hot encode, ignore unseen :contentReference[oaicite:14]{index=14}
])

# ── 5. Combine with ColumnTransformer (by name) ───────────────────────
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_pipeline,     numeric_features),       # name‐based selection :contentReference[oaicite:15]{index=15}
    ("cat", categorical_pipeline, categorical_features),
])

# ── 6. Wrap in final Pipeline with a model ───────────────────────────
pipeline = Pipeline([
    ("prep",  preprocessor),
    ("model", GradientBoostingClassifier(random_state=42)),
])



# Feature Engineering

In [ ]:
from sklearn.compose import ColumnTransformer

# 1. fit only the preprocessing
preprocessor.fit(X)

# 2. transform your raw data
X_prep = preprocessor.transform(X)

# 3. grab the new feature names
feature_names = preprocessor.get_feature_names_out()

# 4. wrap in a DataFrame for easy inspection
import pandas as pd
df_prep = pd.DataFrame(X_prep, columns=feature_names)

num_cols = ["prep__num__" + f for f in numeric_features]
for col in num_cols:
    plt.figure()
    df_prep[col].hist(bins=30)
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.show()

# 5. Correlation matrix heatmap
corr = df_prep[num_cols + ["prep__cat__gift_subs_bool"]].corr()
plt.figure(figsize=(8, 6))
plt.imshow(corr, aspect="auto")
plt.colorbar()
plt.xticks(range(len(corr)), corr.columns, rotation=90)
plt.yticks(range(len(corr)), corr.columns)
plt.title("Feature Correlation")
plt.tight_layout()
plt.show()


# Next steps

In [ ]:
# columns that were tagged as categorical
bad_cols = [
    c for c in categorical_features
    if X_train[c].apply(lambda v: isinstance(v, (list, set, dict))).any()
]
print("Problem columns:", bad_cols)


In [ ]:
from sklearn.model_selection import GridSearchCV

# 1. Define your parameter grid, using "<step_name>__<param_name>"
param_grid = {
    # model hyper-parameters
    "model__n_estimators":    [100, 200, 300],
    "model__learning_rate":   [0.2, 0.5, 1.0],
    "model__max_depth":       [5, 7, 10],

    # (optional) preprocessing hyper-parameters
    "prep__num__impute__strategy": ["median", "mean"],
    "prep__cat__encode__sparse_output":   [True, False],
}

# 2. Wrap your pipeline in GridSearchCV
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,                   # e.g. 5-fold cross-validation
    scoring="roc_auc",      # or whatever metric makes sense
    n_jobs=-1,              # parallelize across all cores
    verbose=1,
)

# 3. Fit on your training data
grid.fit(X_train, y_train)

# 4. Inspect best parameters and performance
print("Best CV score:", grid.best_score_)
# print("Best params:",   grid.best_params_)
for p in grid.best_params_:
    print(f'{p}: {grid.best_params_[p]}')

# 2. Grab the refit pipeline (already fitted on ALL of X_train/y_train)
model = grid.best_estimator_

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:

# ── 8. Fit & evaluate **inside** a config_context block ───────────────
with config_context(enable_metadata_routing=False):
    model.fit(X_train, y_train)                        # no more feature_names_in_ errors :contentReference[oaicite:17]{index=17}

val_pred  = model.predict(X_val)
test_pred = model.predict(X_test)

print("Validation MAE:", mean_absolute_error(y_val,  val_pred))
print("Test MAE:      ", mean_absolute_error(y_test, test_pred))

# ── 9. (Optional) Persist the pipeline ────────────────────────────────
import joblib
joblib.dump(model, "daily_stats_model.pkl")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

In [ ]:
y_proba = model.predict_proba(X_val)[:,1]  # for the positive class
from sklearn.metrics import roc_auc_score
print("ROC AUC:", roc_auc_score(y_val, y_proba))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 1. Get your predictions
y_pred = model.predict(X_val)

# 2. Compute the confusion matrix
labels = model.named_steps['model'].classes_  # for Display’s tick labels
cm = confusion_matrix(y_val, y_pred, labels=labels)

# 3. Plot it
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
_ = disp.plot()          # single Matplotlib figure
plt.title("Confusion Matrix")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.tight_layout()
plt.show()


In [ ]:
# 1. Select your numeric DataFrame
numeric_df = df.select_dtypes(include=['int64', 'float64'])

# 2. Compute correlation matrix
corr = numeric_df.corr()

# 3. Plot it with a bigger figure
fig, ax = plt.subplots(figsize=(10, 8))    # ← larger figure
im = ax.imshow(corr.values, interpolation='nearest')
ticks = np.arange(len(corr.columns))
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(corr.columns, rotation=90)
ax.set_yticklabels(corr.index)
fig.colorbar(im, ax=ax)
plt.title("Feature Correlation Heatmap")
plt.tight_layout()
plt.show()